# Modelo de reconocimiento de letras

Por medio del uso del dataset que se encuentra en [keggle](https://www.kaggle.com/datasets/landlord/handwriting-recognition?resource=download), se desea construir un modelo de inteligencia artificial que proporcione la capacidad de analizar entrada de escritura a mano por medio de una entrada por mouse (Para esto se usará tensorflow.js que permite exportar el modelo para usarlo en el navegador de forma sencilla).

## Entrenamiento de dígitos

Para poder entrenar el modelo en el reconocimiento de letras individuales se usará la base de datos [MNIST](https://docs.ultralytics.com/es/datasets/classify/mnist/) lo cual nos ayudará a posteriormente entrenar el model en identificar las letras de cada uno de las palabras.

In [1]:
import os, sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.python.keras as tfks
from tensorflow.python.keras import utils, layers
import tensorflow_datasets as tfds
from infrastructure.modelo_implementation import modelo_implementation as modelo
DATASET_Jonas = "dataset/archive"
DATASET_MNIST = "dataset/"

2024-04-22 17:02:46.930942: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 17:02:46.934885: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-22 17:02:46.974711: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 17:02:48.338096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jonasaray/Documents/UCAB/Decimo/Inteligencia artificial/Proyecto/LetterRecognition/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.htm

In [2]:
# Modelo
m = tfks.Sequential(layers=[
    layers.Flatten(input_shape=(28,28,1)),
    layers.Dense(50, activation=tf.nn.relu),
    layers.Dense(50, activation=tf.nn.relu),
    layers.Dense(27, activation=tf.nn.softmax)])

In [3]:
sys.setrecursionlimit(10000)
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  1.70 url/s]
Extraction completed...: 0 file [00:02, ? file/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:02<00:00,  1.48 url/s]


RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
ds_train = ds_train.map(modelo.imageToCorrectSize(), )

In [10]:
train = pd.read_csv(os.path.join(DATASET_Jonas, "written_name_train_v2.csv"), header=None)
train.iloc

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE
...,...,...
330956,TRAIN_330957.jpg,LENNY
330957,TRAIN_330958.jpg,TIFFANY
330958,TRAIN_330959.jpg,COUTINHO DESA
330959,TRAIN_330960.jpg,MOURAD


In [9]:
test = pd.read_csv(os.path.join(DATASET_Jonas, "written_name_test_v2.csv"), header=None)
test

,0,1
0,FILENAME,IDENTITY
1,TEST_0001.jpg,KEVIN
2,TEST_0002.jpg,CLOTAIRE
3,TEST_0003.jpg,LENA
4,TEST_0004.jpg,JULES
...,...,...
41366,TEST_41366.jpg,ALEXANE
41367,TEST_41367.jpg,PEREIRA-SILVA
41368,TEST_41368.jpg,LAURENT
41369,TEST_41369.jpg,DEFFENSE


In [11]:
validation = pd.read_csv(os.path.join(DATASET_Jonas, "written_name_validation_v2.csv"), header=None)
validation

,FILENAME,IDENTITY
0,VALIDATION_0001.jpg,BILEL
1,VALIDATION_0002.jpg,LAUMIONIER
2,VALIDATION_0003.jpg,LEA
3,VALIDATION_0004.jpg,JEAN-ROCH
4,VALIDATION_0005.jpg,RUPP
...,...,...
41365,VALIDATION_41366.jpg,CHAILLAN
41366,VALIDATION_41367.jpg,BAROUH
41367,VALIDATION_41368.jpg,MAXENCE
41368,VALIDATION_41369.jpg,HAMELIN
